In [ ]:
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage,OdaLightCurve
from oda_api.data_products import BinaryData
import os
from astropy.io import fits
import numpy as np
from numpy import sqrt,exp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
source_name='NGC 2110'
ra=88.047400
dec=-7.456247
radius=15.
radius_jemx=5.
Tstart='2003-03-16T00:00:00'
Tstop='2011-04-06T23:59:59'
host='https://www.astro.unige.ch/cdci/astrooda/dispatch-data'
time_bin=100
E1_keV=20.
E2_keV=40.
E1_keV_jemx=3.
E2_keV_jemx=20.

rebin=10

In [ ]:
disp=DispatcherAPI(host=host)

In [ ]:
import requests
url="https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw/timesystem/api/v1.0/scwlist/cons/"
def queryxtime(**args):
    params=Tstart+'/'+Tstop+'?&ra='+str(ra)+'&dec='+str(dec)+'&radius='+str(radius)+'&min_good_isgri=1000'
    print(url+params)
    return requests.get(url+params).json()
def queryxtime_jemx1(**args):
    params=Tstart+'/'+Tstop+'?&ra='+str(ra)+'&dec='+str(dec)+'&radius='+str(radius_jemx)+'&min_good_jemx1=1000'
    print(url+params)
    return requests.get(url+params).json()
def queryxtime_jemx2(**args):
    params=Tstart+'/'+Tstop+'?&ra='+str(ra)+'&dec='+str(dec)+'&radius='+str(radius_jemx)+'&min_good_jemx2=1000'
    print(url+params)
    return requests.get(url+params).json()



In [ ]:
scwlist=queryxtime()
m=len(scwlist)
pointings_osa10=[]
pointings_osa11=[]
for i in range(m):
    if scwlist[i][-2:]=='10':
        if(int(scwlist[i][:4])<1626):
            pointings_osa10.append(scwlist[i]+'.001')
        else:
            pointings_osa11.append(scwlist[i]+'.001')
m_osa10=len(pointings_osa10)
m_osa11=len(pointings_osa11)
print(pointings_osa10)
print(pointings_osa11)

In [ ]:
scw_lists_osa10=[]
scw_lists_osa11=[]
count=0
scw_string=''
for i in range(m_osa10):
    if count<50:
        scw_string=scw_string+str(pointings_osa10[i])+','
        count+=1
    else:
        scw_lists_osa10.append(scw_string[:-1])
        count=1
        scw_string=str(pointings_osa10[i])+','
scw_lists_osa10.append(scw_string[:-1])
print(len(scw_lists_osa10))
count=0
scw_string=''
for i in range(m_osa11):
    if count<50:
        scw_string=scw_string+str(pointings_osa11[i])+','
        count+=1
    else:
        scw_lists_osa11.append(scw_string[:-1])
        count=1
        scw_string=str(pointings_osa11[i])+','
scw_lists_osa11.append(scw_string[:-1])
print(len(scw_lists_osa11))



In [ ]:
data=disp.get_product(instrument='isgri',
                      product='isgri_image',
                      scw_list=scw_lists_osa10[0],
                      E1_keV=E1_keV,
                      E2_keV=E2_keV,
                      osa_version='OSA10.2',
                      RA=ra,
                      DEC=dec,
                      detection_threshold=5,
                      product_type='Real')

In [ ]:
data.dispatcher_catalog_1.table

In [ ]:
api_cat=data.dispatcher_catalog_1.get_api_dictionary()

In [ ]:
spectrum_results=[]
for i in range(len(scw_lists_osa10)):
    print(i,pointings_osa10[i])
    data=disp.get_product(instrument='isgri',
                        product='isgri_spectrum',
                        scw_list=scw_lists_osa10[i],
                        query_type='Real',
                        osa_version='OSA10.2',
                        RA=ra,
                        DEC=dec,
                        product_type='Real',
                        selected_catalog=api_cat)
    spectrum_results.append(data)

In [ ]:
d=spectrum_results[0]
for ID,s in enumerate(d._p_list):
    if (s.meta_data['src_name']==source_name):
        if(s.meta_data['product']=='isgri_spectrum'):
            ID_spec=ID
        if(s.meta_data['product']=='isgri_arf'):
            ID_arf=ID
        if(s.meta_data['product']=='isgri_rmf'):
            ID_rmf=ID

print(ID_spec, ID_arf, ID_rmf)

In [ ]:
d=spectrum_results[0]
spec=d._p_list[ID_spec].data_unit[1].data
arf=d._p_list[ID_arf].data_unit[1].data
rmf=d._p_list[ID_rmf].data_unit[2].data
ch=spec['CHANNEL']
rate=spec['RATE']*0.
err=spec['STAT_ERR']*0.
syst=spec['SYS_ERR']*0.
rate.fill(0)
err.fill(0)
syst.fill(0)
qual=spec['QUALITY']
matrix=rmf['MATRIX']*0.
specresp=arf['SPECRESP']*0.
tot_expos=0.
corr_expos=np.zeros(len(rate))
print(len(rate))
for k in range(len(scw_lists_osa10)):
    d=spectrum_results[k]
    spec=d._p_list[ID_spec].data_unit[1].data
    arf=d._p_list[ID_arf].data_unit[1].data
    rmf=d._p_list[ID_rmf].data_unit[2].data
    expos=d._p_list[0].data_unit[1].header['EXPOSURE']
    tot_expos=tot_expos+expos
    print(k,expos)
    for j in range(len(rate)):
        if(spec['QUALITY'][j]==0):          
            rate[j]=rate[j]+spec['RATE'][j]/(spec['STAT_ERR'][j])**2
            err[j]=err[j]+1./(spec['STAT_ERR'][j])**2
            syst[j]=syst[j]+(spec['SYS_ERR'][j])**2*expos
            corr_expos[j]=corr_expos[j]+expos
    matrix=matrix+rmf['MATRIX']*expos
    specresp=specresp+arf['SPECRESP']*expos

for i in range(len(rate)):
    if err[i]>0.:
        rate[i]=rate[i]/err[i]
        err[i]=1./sqrt(err[i])
matrix=matrix/tot_expos
specresp=specresp/tot_expos
syst=sqrt(syst/(corr_expos+1.))
print('Total exposure:',tot_expos)

In [ ]:
d._p_list[ID_spec].data_unit[1].data['RATE']=rate
d._p_list[ID_spec].data_unit[1].data['STAT_ERR']=err
d._p_list[ID_rmf].data_unit[2].data['MATRIX']=matrix
d._p_list[ID_arf].data_unit[1].data['SPECRESP']=specresp

In [ ]:
name=source_name.replace(" ", "")
specname=name+'_spectrum_osa10.fits'
arfname=name+'_arf_osa10.fits.gz'
rmfname=name+'_rmf_osa10.fits.gz'
d._p_list[ID_spec].write_fits_file(specname)
d._p_list[ID_arf].write_fits_file(arfname)
d._p_list[ID_rmf].write_fits_file(rmfname)

In [ ]:
hdul = fits.open(specname, mode='update')
hdr=hdul[1].header
hdr.set('EXPOSURE', tot_expos)
hdul.close()

In [ ]:
#if token!='':
scwlist_jemx1=queryxtime_jemx1()
scwlist_jemx2=queryxtime_jemx2()

m=len(scwlist_jemx1)
pointings_osa10_jemx1=[]
pointings_osa11_jemx1=[]
for i in range(m):
    if scwlist_jemx1[i][-2:]=='10':
        if(int(scwlist_jemx1[i][:4])<1626):
            pointings_osa10_jemx1.append(scwlist_jemx1[i]+'.001')
        else:
            pointings_osa11_jemx1.append(scwlist_jemx1[i]+'.001')
m_osa10_jemx1=len(pointings_osa10_jemx1)
m_osa11_jemx1=len(pointings_osa11_jemx1)
m=len(scwlist_jemx2)
pointings_osa10_jemx2=[]
pointings_osa11_jemx2=[]
for i in range(m):
    if scwlist_jemx2[i][-2:]=='10':
        if(int(scwlist_jemx2[i][:4])<1626):
            pointings_osa10_jemx2.append(scwlist_jemx2[i]+'.001')
        else:
            pointings_osa11_jemx2.append(scwlist_jemx2[i]+'.001')
m_osa10_jemx2=len(pointings_osa10_jemx2)
m_osa11_jemx2=len(pointings_osa11_jemx2)

print(pointings_osa10_jemx1)
print(pointings_osa11_jemx1)

In [ ]:
scw_lists_osa10_jemx1=[]
scw_lists_osa11_jemx1=[]
count=0
scw_string=''
for i in range(m_osa10_jemx1):
    if count<50:
        scw_string=scw_string+str(pointings_osa10_jemx1[i])+','
        count+=1
    else:
        scw_lists_osa10_jemx1.append(scw_string[:-1])
        count=1
        scw_string=str(pointings_osa10_jemx1[i])+','
scw_lists_osa10_jemx1.append(scw_string[:-1])
print(len(scw_lists_osa10_jemx1))
count=0
scw_string=''
for i in range(m_osa11_jemx1):
    if count<50:
        scw_string=scw_string+str(pointings_osa11_jemx1[i])+','
        count+=1
    else:
        scw_lists_osa11_jemx1.append(scw_string[:-1])
        count=1
        scw_string=str(pointings_osa11_jemx1[i])+','
scw_lists_osa11_jemx1.append(scw_string[:-1])
print(len(scw_lists_osa11_jemx1))

scw_lists_osa10_jemx2=[]
scw_lists_osa11_jemx2=[]
count=0
scw_string=''
for i in range(m_osa10_jemx2):
    if count<50:
        scw_string=scw_string+str(pointings_osa10_jemx2[i])+','
        count+=1
    else:
        scw_lists_osa10_jemx2.append(scw_string[:-1])
        count=1
        scw_string=str(pointings_osa10_jemx2[i])+','
scw_lists_osa10_jemx2.append(scw_string[:-1])
print(len(scw_lists_osa10_jemx2))
count=0
scw_string=''
for i in range(m_osa11_jemx2):
    if count<50:
        scw_string=scw_string+str(pointings_osa11_jemx2[i])+','
        count+=1
    else:
        scw_lists_osa11_jemx2.append(scw_string[:-1])
        count=1
        scw_string=str(pointings_osa11_jemx2[i])+','
scw_lists_osa11_jemx2.append(scw_string[:-1])
print(len(scw_lists_osa11_jemx2))



In [ ]:
data=disp.get_product(instrument='jemx',
                      jemx_num=1,
                      product='jemx_image',
                      scw_list=scw_lists_osa10_jemx1[0],
                      E1_keV=E1_keV_jemx,
                      E2_keV=E2_keV_jemx,
                      osa_version='OSA10.2',
                      RA=ra,
                      DEC=dec,
                      detection_threshold=5,
                      product_type='Real')

In [ ]:
data.dispatcher_catalog_1.table

In [ ]:
api_cat_jemx=data.dispatcher_catalog_1.get_api_dictionary()

In [ ]:
spectrum_results_jemx=[]
for i in range(len(scw_lists_osa10_jemx1)):
    data=disp.get_product(instrument='jemx',
                        jemx_num=1,
                        product='jemx_spectrum',
                        scw_list=scw_lists_osa10_jemx1[i],
                        query_type='Real',
                        osa_version='OSA10.2',
                        RA=ra,
                        DEC=dec,
                        product_type='Real',
                        selected_catalog=api_cat_jemx)
    spectrum_results_jemx.append(data)


In [ ]:
for ID,s in enumerate(spectrum_results_jemx[0]._p_list):
    print(s.meta_data)
    if (s.meta_data['src_name']==source_name):
        if(s.meta_data['product']=='jemx_spectrum'):
            ID_spec_jemx1=ID
        if(s.meta_data['product']=='jemx_arf'):
            ID_arf_jemx1=ID
        if(s.meta_data['product']=='jemx_rmf'):
            ID_rmf_jemx1=ID

print(ID_spec_jemx1, ID_arf_jemx1, ID_rmf_jemx1)



In [ ]:
d=spectrum_results_jemx[0]
spec=d._p_list[ID_spec_jemx1].data_unit[1].data
arf=d._p_list[ID_arf_jemx1].data_unit[1].data
rmf=d._p_list[ID_rmf_jemx1].data_unit[2].data
ch=spec['CHANNEL']
rate_jemx1=spec['RATE']*0.
err_jemx1=spec['STAT_ERR']*0.
syst_jemx1=spec['SYS_ERR']*0.
rate_jemx1.fill(0)
err_jemx1.fill(0)
syst_jemx1.fill(0)
qual=spec['QUALITY']
matrix=rmf['MATRIX']*0.
specresp=arf['SPECRESP']*0.
tot_expos_jemx1=0.
corr_expos_jemx1=np.zeros(len(rate_jemx1))
print(len(rate_jemx1))
for k in range(len(scw_lists_osa10_jemx1)):
    d=spectrum_results_jemx[k]
    spec=d._p_list[ID_spec_jemx1].data_unit[1].data
    arf=d._p_list[ID_arf_jemx1].data_unit[1].data
    rmf=d._p_list[ID_rmf_jemx1].data_unit[2].data
    expos=d._p_list[0].data_unit[1].header['EXPOSURE']
    tot_expos_jemx1=tot_expos_jemx1+expos
    print(k,expos)
    for j in range(len(rate_jemx1)):
        if(spec['QUALITY'][j]==0):          
            rate_jemx1[j]=rate_jemx1[j]+spec['RATE'][j]/(spec['STAT_ERR'][j])**2
            err_jemx1[j]=err_jemx1[j]+1./(spec['STAT_ERR'][j])**2
            syst_jemx1[j]=syst_jemx1[j]+(spec['SYS_ERR'][j])**2*expos
            corr_expos_jemx1[j]=corr_expos_jemx1[j]+expos
    matrix=matrix+rmf['MATRIX']*expos
    specresp=specresp+arf['SPECRESP']*expos

for i in range(len(rate_jemx1)):
    if err_jemx1[i]>0.:
        rate_jemx1[i]=rate_jemx1[i]/err_jemx1[i]
        err_jemx1[i]=1./sqrt(err_jemx1[i])
matrix=matrix/tot_expos_jemx1
specresp=specresp/tot_expos_jemx1
syst_jemx1=sqrt(syst_jemx1/(corr_expos_jemx1+1.))
print('Total exposure:',tot_expos_jemx1)

In [ ]:
d._p_list[ID_spec_jemx1].data_unit[1].data['RATE']=rate_jemx1
d._p_list[ID_spec_jemx1].data_unit[1].data['STAT_ERR']=err_jemx1
d._p_list[ID_rmf_jemx1].data_unit[2].data['MATRIX']=matrix
d._p_list[ID_arf_jemx1].data_unit[1].data['SPECRESP']=specresp
name=source_name.replace(" ", "")
specname=name+'_spectrum_osa10_jemx1.fits'
arfname=name+'_arf_osa10_jemx1.fits.gz'
rmfname=name+'_rmf_osa10_jemx1.fits.gz'
d._p_list[ID_spec_jemx1].write_fits_file(specname)
d._p_list[ID_arf_jemx1].write_fits_file(arfname)
d._p_list[ID_rmf_jemx1].write_fits_file(rmfname)

In [ ]:
!./spectrum_fit_isgri_jemx_osa10.sh $name $rebin

In [ ]:
golden_ratio=0.5*(1.+np.sqrt(5))
width=4.
plt.figure(figsize=(golden_ratio*width,width))
alpha=1
linewidth=4
fontsize=11

spectrum=np.genfromtxt(name+'_spectrum_osa10.txt',skip_header=3)
en=spectrum[:,0]
en_err=spectrum[:,1]
fl=spectrum[:,2]
fl_err=spectrum[:,3]
mo=spectrum[:,4]
plt.errorbar(en,fl,xerr=en_err,yerr=fl_err,linestyle='none',linewidth=linewidth,color='black',alpha=alpha)
plt.plot(en,mo,color='black',linewidth=linewith/2)


# factor=0.9
# d=np.genfromtxt('NGC2110_isgri_lubinski.csv')
# en=d[:,0]
# f=d[:,1]
# fmax=d[:,2]
# fmin=d[:,3]
# plt.errorbar(en,factor*f,yerr=[factor*(f-fmin),factor*(fmax-f)],linestyle='none',linewidth=4,color='blue',label=r'$0.9\times$ Lubinski et al. (2016)')

# d=np.genfromtxt('NGC2110_model_lubinski.csv')
# en=d[:,0]
# f=d[:,1]
# plt.plot(en,factor*f,color='blue',alpha=0.5)



plt.tick_params(axis='both', which='major', labelsize=fontsize)
plt.xscale('log')
plt.yscale('log')
#plt.ylim(1.e-1,3.e2)
#plt.xlim(3,500)
plt.legend(loc='lower left',fontsize=fontsize)

plt.xlabel('$E$ [keV]',fontsize=fontsize)
plt.ylabel('$E^2F_E$ [keV$\,$cm$^{-2}\,\mathrm{s}^{-1}$]',fontsize=fontsize)

#plt.savefig(name+'_spectra.pdf',format='pdf',dpi=100)

plt.savefig('NGC2110_Lubinsky_2016_spectrum.pdf', format='pdf',dpi=160)
